In [1]:
knitr::opts_chunk$set(echo = TRUE)
library(ranger)
library(caret)
library(readr)
library(rpart)
library(glmnet)

df <- read.csv("adult.csv", header = FALSE, na.strings = c(" ?", "NA", "."))
names(df) = c("age", "workclass", "fnlwgt", "education", "educationNum", "maritalStatus", "occupation", "relationship", "race", "sex", "capitalGain", "capitalLoss", "hoursPerWeek", "nativeCountry", "target")
df$age <- as.numeric(df$age)
df$workclass <- as.factor(df$workclass)
df$fnlwgt <- as.numeric(df$fnlwgt)
df$education <- as.factor(df$education)
df$educationNum <- as.numeric(df$educationNum)
df$maritalStatus <- as.factor(df$maritalStatus)
df$occupation <- as.factor(df$occupation)
df$relationship <- as.factor(df$relationship)
df$race <- as.factor(df$race)
df$sex <- as.factor(df$sex)
df$capitalGain <- as.numeric(df$capitalGain)
df$capitalLoss <- as.numeric(df$capitalLoss)
df$hoursPerWeek <- as.numeric(df$hoursPerWeek)
df$nativeCountry <- as.factor(df$nativeCountry)

repnum = function(x){
  for(i in 1:ncol(x)){
    if(is.numeric(x[,i])){
     x[is.na(x[,i]), i] <- mean(x[,i], na.rm = TRUE)
    }
  }
  return(x)
}
df = repnum(df)

repcat = function(x){
  for(i in 1:ncol(x)){
    if(is.factor(x[,i])){
     x[,i][is.na(x[,i])] = levels(x[,i])[which.max(table(x[,i]))]
    }
  }
  return(x)
}
df = repcat(df)

levels(df$education) = c("HS", "HS", "HS", "Ele", "Middle", "Middle", "HS", "UGrad", "UGrad", "UGrad", "PGrad", "HS", "PGrad", "Ele", "UGrad", "UGrad")
levels(df$occupation) = c("WCollar", "Military", "BCollar", "WCollar", "BCollar", "BCollar", "BCollar", "Other", "BCollar", "WCollar", "BCollar", "WCollar", "WCollar", "BCollar")
levels(df$nativeCountry) = c("Asia", "NAmerica", "Asia", "SAmerica", "CAmerica", "CAmerica", "SAmerica", "SAmerica", "Europe", "Europe", "Europe", "Europe", "CAmerica", "CAmerica", "Europe", "CAmerica", "Asia", "Europe", "Asia", "Asia", "Europe", "Europe", "CAmerica", "Asia", "Asia", "NAmerica", "CAmerica", "NAmerica", "SAmerica", "Asia", "Europe", "Europe", "CAmerica", "Europe", "Asia", "Asia", "Asia", "SAmerica", "NAmerica", "Asia", "Europe")
levels(df$workclass) = c("FederalGov", "LocalGov", "NeverWorked", "Private", "SelfEmpInc", "SelfEmpNotInc", "StateGov", "WithoutPay")
levels(df$maritalStatus) = c("Divorved", "MarriedAFSpouse", "MarriedCivSpouse", "MarriedSpouseAbsent", "NeverMarried", "Separated", "Widowed")
levels(df$relationship) = c("Husband", "NotInFamily", "OtherRelative", "OwnChild", "Unmarried", "Wife")
levels(df$race) = c("AmerIndianEskimo", "AsianPacIslander", "Black", "Other", "White")
levels(df$target) = c("Less50k", "More50k")

myGrid <- expand.grid(mtry = 2, splitrule = c("gini"), min.node.size = c(1:3))
model <- train(target~.,data = df, method = "ranger", 
               trControl = trainControl(method ="cv", number = 7, verboseIter = TRUE),
               tuneGrid = myGrid)
model

splitIndex <- createDataPartition(df$target, p = .70, list = FALSE, times = 1)
train <- df[ splitIndex,]
test <- df[-splitIndex,]
trctrl <- trainControl(method = "repeatedcv", number = 10, repeats = 3)
dtree <- train(target ~., data = train, method = "rpart",
                   parms = list(split = "information"),
                   trControl=trctrl,
                   tuneLength = 10)
dtree
plot(dtree)

plot(model)

myGrid2 <- expand.grid(mtry = 5, splitrule = c("gini","extratrees"), min.node.size = c(1:5))
model2 <- train(target~.,data = df, method = "ranger", 
               trControl = trainControl(method ="cv", number = 7, verboseIter = TRUE),
               tuneGrid = myGrid2)
model2

glmcntrl <- trainControl(method = "cv", number = 10, summaryFunction = twoClassSummary, classProbs = TRUE, verboseIter = TRUE)
myGrid3 <- expand.grid(alpha = 0:1, lambda = seq(0.0001, 1, length = 10))
model3 <- train(target ~ ., data = df, tuneGrid = myGrid3, method = "glmnet", trControl = glmcntrl)
model3

Loading required package: lattice
Loading required package: ggplot2
Loading required package: Matrix
Loading required package: foreach
Loaded glmnet 2.0-16

Warning message in file(file, "rt"):
"cannot open file 'adult.csv': No such file or directory"

ERROR: Error in file(file, "rt"): cannot open the connection


In [2]:
knitr::opts_chunk$set(echo = TRUE)
library(ranger)
library(caret)
library(readr)
library(rpart)
library(glmnet)

df <- read.csv("titanic.csv")
df$Survived <- as.factor(df$Survived)
df$Pclass <- as.factor(df$Pclass)
df$SibSp <- as.numeric(df$SibSp)
df$Parch <- as.numeric(df$Parch)

levels(df$Survived) = c("dnSurvive", "Survive")

df$PassengerId <- NULL
df$Name <- NULL
df$Ticket <- NULL
df$Cabin <- NULL

repnum = function(x){
  for(i in 1:ncol(x)){
    if(is.numeric(x[,i])){
     x[is.na(x[,i]), i] <- mean(x[,i], na.rm = TRUE)
    }
  }
  return(x)
}
df = repnum(df)

repcat = function(x){
  for(i in 1:ncol(x)){
    if(is.factor(x[,i])){
     x[,i][is.na(x[,i])] = levels(x[,i])[which.max(table(x[,i]))]
    }
  }
  return(x)
}
df = repcat(df)

myGrid <- expand.grid(mtry = 2, splitrule = c("gini"), min.node.size = c(1:3))
model <- train(Survived ~.,data = df, method = "ranger", 
               trControl = trainControl(method ="cv", number = 7, verboseIter = TRUE),
               tuneGrid = myGrid)
model

splitIndex <- createDataPartition(df$Survived, p = .70, list = FALSE, times = 1)
train <- df[ splitIndex,]
test <- df[-splitIndex,]
trctrl <- trainControl(method = "repeatedcv", number = 10, repeats = 3)
dtree <- train(Survived ~., data = train, method = "rpart",
                   parms = list(split = "information"),
                   trControl=trctrl,
                   tuneLength = 10)
dtree
plot(dtree)

plot(model)

myGrid2 <- expand.grid(mtry = 7, splitrule = c("gini","extratrees"), min.node.size = c(1:20))
model2 <- train(Survived ~.,data = df, method = "ranger", 
               trControl = trainControl(method ="cv", number = 7, verboseIter = TRUE),
               tuneGrid = myGrid2)
model2

glmcntrl <- trainControl(method = "cv", number = 10, summaryFunction = twoClassSummary, classProbs = TRUE, verboseIter = TRUE)
myGrid3 <- expand.grid(alpha = 0:1, lambda = seq(0.0001, 1, length = 10))
model3 <- train(Survived ~ ., data = df, tuneGrid = myGrid3, method = "glmnet", trControl = glmcntrl)
model3

Warning message in file(file, "rt"):
"cannot open file 'titanic.csv': No such file or directory"

ERROR: Error in file(file, "rt"): cannot open the connection
